In [ ]:
radar_plot = [ 0.4375, 0.671875, 0.46875, 0.796875, 0.890625, 0.828125, 0.65625,  0.625, 0.703125, 0.703125,
               0.953125, 0.71875, 0.8125, 0.921875, 0.90625, 0.953125, 0.78125, 0.734375, 0.53125, 0.484375,
                 0.671875, 0.53125, 0.828125, 0.828125, 0.265625, 0.703125, 0.6875, 0.609375, 0.9375, 0.796875,
                 0.921875, 0.578125, 0.796875, 0.5, 0.734375, 0.859375, 0.421875, 0.703125, 1.0, 0.28125]


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
PATH_TO_LABELS = 'data/list_attr_celeba.csv'
class_names = open(PATH_TO_LABELS).readlines()[0].split(',')[1:]
df = df = pd.DataFrame(
        {
            'Name': class_names,
            'Value': radar_plot
        })

In [ ]:
##### RADAR PLOT #####
# Libraries
import matplotlib.pyplot as plt
import pandas as pd
from math import pi
 
# Set data
df = pd.DataFrame({
'group': ['A','B','C','D'],
'var1': [38, 1.5, 30, 4],
})
print
# number of variable
categories=list(df)[1:]
N = len(categories)
 
# We are going to plot the first line of the data frame.
# But we need to repeat the first value to close the circular graph:
values=df.loc[0].drop('Name').values.flatten().tolist()
values += values[:1]
values
 
# What will be the angle of each axis in the plot? (we divide the plot / number of variable)
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]
 
# Initialise the spider plot
ax = plt.subplot(111, polar=True)
 
# Draw one axe per variable + add labels
plt.xticks(angles[:-1], categories, color='grey', size=8)
 
# Draw ylabels
ax.set_rlabel_position(0)
plt.yticks([10,20,30], ["10","20","30"], color="grey", size=7)
plt.ylim(0,40)
 
# Plot data
ax.plot(angles, values, linewidth=1, linestyle='solid')
 
# Fill area
ax.fill(angles, values, 'b', alpha=0.1)

# Show the graph
plt.show()

In [ ]:
##### CIRCULAR BARPLOT #####


plt.figure(figsize=(20,10))

# plot polar axis
ax = plt.subplot(111, polar=True)

# remove grid
plt.axis('on')
plt.xlabel('off')
plt.ylabel('off')



# Set the coordinates limits
upperLimit = 1
lowerLimit = 0

# Compute max and min in the dataset
max = df['Value'].max()

# Let's compute heights: they are a conversion of each item value in those new coordinates
# In our example, 0 in the dataset will be converted to the lowerLimit (10)
# The maximum will be converted to the upperLimit (100)
slope = (max - lowerLimit) / max
heights = slope * df.Value + lowerLimit

# Compute the width of each bar. In total we have 2*Pi = 360°
width = 2*np.pi / len(df.index)

# Compute the angle each bar is centered on:
indexes = list(range(1, len(df.index)+1))
angles = [element * width for element in indexes]
angles

# Draw bars
bars = ax.bar(
    x=angles, 
    height=heights, 
    width=width, 
    bottom=lowerLimit,
    linewidth=2, 
    edgecolor="white")

labelPadding = 0

# Add labels
for bar, angle, height, label in zip(bars,angles, heights, df["Name"]):

    # Labels are rotated. Rotation must be specified in degrees :(
    rotation = np.rad2deg(angle)

    # Flip some labels upside down
    alignment = ""
    if angle >= np.pi/2 and angle < 3*np.pi/2:
        alignment = "right"
        rotation = rotation + 180
    else: 
        alignment = "left"

    # Finally add the labels
    ax.text(
        x=angle, 
        y=lowerLimit + bar.get_height() + labelPadding, 
        s=label, 
        ha=alignment, 
        va='center', 
        rotation=rotation, 
        rotation_mode="anchor")